<a href="https://colab.research.google.com/github/Trevorchenmsu/Facebook-Hateful-Memes-Challenge/blob/main/V2_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install tensorboard package
!pip install tensorboard

In [ ]:
# check the files in home folder
import os
import numpy
import torch
from torch.utils import tensorboard
os.environ['MKL_THREADING_LAYER'] = 'GNU'
os.environ['OC_DISABLE_DOT_ACCESS_WARNING']='1'
home = "/home/jishengchenmsu"
os.chdir(home)
os.getcwd()
# # check the number of images
!ls /home/jishengchenmsu 

FB_V0.ipynb	       features_101		  images_folder
X-152-32x8d-IN5k.pkl   features_grid_extract	  imgs_features_download.ipynb
anaconda3	       features_grid_extract.zip  mmf
feature_extract.ipynb  grid-feats-vqa		  vqa-maskrcnn-benchmark


In [2]:
# Install specified versions of `torch` and `torchvision`, before installing mmf (causes an issue)
!pip install torch==1.6.0 torchvision==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
# Install MMF from source. Clone the following repo where mmf does not 
# install default image features, since we will use our own features
!git clone --branch no_feats --config core.symlinks=true https://github.com/rizavelioglu/mmf.git

In [4]:
# install mmf
os.chdir(os.path.join(home, "mmf"))
!pip install --editable .
!ls /home/jishengchenmsu/mmf

In [5]:
# install gdonw so that we can download files from google drive
!pip install gdown

In [ ]:
# download the zip file from the link, but it doesn not work
img_orc_dir = "images_folder"
os.chdir(os.path.join(home, img_orc_dir))
!gdown https://drive.google.com/uc?id=1fs4wH5M51Zjs1IDRXGxLMPDtlYDhy9iT

Downloading...
From: https://drive.google.com/uc?id=1fs4wH5M51Zjs1IDRXGxLMPDtlYDhy9iT
To: /home/jishengchenmsu/images_folder/img_ocr.zip
3.65GB [00:24, 152MB/s] 


In [ ]:
# convert the zip file under this path
import os
img_orc_dir = "images_folder"
os.chdir(os.path.join(home, img_orc_dir))
os.environ['MKL_THREADING_LAYER'] = 'GNU'
!mmf_convert_hm --zip_file="img_ocr.zip"  --password="" --bypass_checksum 1

Data folder is /home/jishengchenmsu/.cache/torch/mmf/data
Zip path is img_ocr.zip
Copying img_ocr.zip
Unzipping img_ocr.zip
Extracting the zip can take time. Sit back and relax.
Moving train.jsonl
Moving dev_seen.jsonl
Moving test_seen.jsonl
Moving dev_unseen.jsonl
Moving test_unseen.jsonl
Moving img


In [ ]:
# check the files under the cach folder
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations

# # check the number of images
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ |wc -l
print("Done")

data  img  img_ocr.zip	img_raw.zip
dev_seen.jsonl	  test_seen.jsonl    train.jsonl
dev_unseen.jsonl  test_unseen.jsonl
12140
Done


In [ ]:
# Free up the disk by removing .zip, .tar files
# !rm -rf /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img_ocr.zip

In [ ]:
# check the files under the cach folder
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations

# # check the number of images
!ls /home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ |wc -l

annotations  extras  extras.tar.gz  features  images
data  img  img_ocr.zip	img_raw.zip
dev_seen.jsonl	  test_seen.jsonl    train.jsonl
dev_unseen.jsonl  test_unseen.jsonl
12140


In [ ]:
# number of unique images in the dataset
import pandas as pd
import os

annotation_dir = "/home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations"
img_dir = "/home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/"

# Collect all the annotations (from Phase-2)
train       = pd.read_json(f"{annotation_dir}/train.jsonl", lines=True)
dev_seen    = pd.read_json(f"{annotation_dir}/dev_seen.jsonl", lines=True)
dev_unseen  = pd.read_json(f"{annotation_dir}/dev_unseen.jsonl", lines=True)
test_seen   = pd.read_json(f"{annotation_dir}/test_seen.jsonl", lines=True)
test_unseen = pd.read_json(f"{annotation_dir}/test_unseen.jsonl", lines=True)

# Create 2 sets: 
#   A set of strings, 'a': for all the image names,
#   A set of lists, 'b': for all the image names in dataset, e.g. train, dev_seen, etc.
a = os.listdir(f"{img_dir}")
b = []
for i in [train, dev_seen, dev_unseen, test_seen, test_unseen]:
    b.append(list(i["img"].str.split("/").str.get(1)))

set_mapping = ['train', 'dev_seen', 'dev_unseen', 'test_seen', 'test_unseen']
total_size = 0
print("#of images in: ")
for idx, i in enumerate(b):
    total_size += len(set(i))
    print(f"\t'{set_mapping[idx]}'  \t:", len(set(i)))
else:
    print(f"\nIn total there are {total_size} images,",
          "\nBut the # of images in /img/ directory is: ", len(a))

#of images in: 
	'train'  	: 8500
	'dev_seen'  	: 500
	'dev_unseen'  	: 540
	'test_seen'  	: 1000
	'test_unseen'  	: 2000

In total there are 12540 images, 
But the # of images in /img/ directory is:  12140


In [6]:
# extract image features， intall packages and repos
import os
home = "/home/jishengchenmsu"
os.environ['MKL_THREADING_LAYER'] = 'GNU'
os.chdir(home)
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark

!pip install ninja yacs cython matplotlib

os.chdir(os.path.join(home, "vqa-maskrcnn-benchmark"))
!rm -rf build
!python setup.py build develop


In [ ]:
# install cv2
!pip install opencv-python

In [7]:
import os
os.chdir(os.path.join(home, "mmf/tools/scripts/features/"))
out_folder = os.path.join(home, "features_101/")
print("starting extracting")
!python extract_features_vmb.py --config_file "https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model.yaml" \
                                --model_name "X-101" \
                                --output_folder $out_folder \
                                --image_dir "/home/jishengchenmsu/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/" \
                                --num_features 100 \
                                --start_index 0 \
                                --end_index 12140 \
                                # --exclude_list "/content/exclude.txt"
                                # --feature_name "fc6" \
                                # --confidence_threshold 0. \

In [8]:
# !ls /home/jishengchenmsu/features_101/ |wc -l
!zip -r features_101.zip /home/jishengchenmsu/features_101

In [ ]:

# ****************** download feature data ********************
os.chdir(home)

# features after OCR (Maks RCNN + Resnet152)
# !gdown https://drive.google.com/uc?id=13-VeoX4eYQH-PAlC5SQXO45bduBXMmri

# features from 3rd place (no OCR, Maks RCNN + Resnet152)
# !gdown https://drive.google.com/uc?id=1qlCkuzOVZJl8JRrfD-HcjWG12sQs3RYe

# features after OCR (grid region extraction)
!gdown https://drive.google.com/uc?id=17GXc6Zz03QCk82J3acEI3AWIVEPuTzpk

Downloading...
From: https://drive.google.com/uc?id=17GXc6Zz03QCk82J3acEI3AWIVEPuTzpk
To: /home/jishengchenmsu/features_grid_extract.zip
12.7MB [00:00, 79.4MB/s]


In [9]:
# Unzip OCR feature (mask RCNN + Resnet152)
# !unzip features_ocr.zip -d /home/jishengchenmsu/

# unzip OCR feature (grid region extraction)
!unzip features_grid_extract.zip -d /home/jishengchenmsu/features_grid_extract/

In [ ]:
# Free up the disk by removing .zip, .tar files
# !rm -rf /home/jishengchenmsu/feature_ocr/features_ocr.zipcd 

In [ ]:
# Check how many images we have in total
# !ls /home/jishengchenmsu/features_ocr/ |wc -l
# !ls /home/jishengchenmsu/features_grid_extract/hm2021/ |wc -l
# !ls /home/jishengchenmsu/features_grid_extract/hm2021
import numpy as np
data = np.load("/home/jishengchenmsu/features_grid_extract/hm2021/01247.npy", allow_pickle=True)
data.item(0)
data.item(0)["bbox"].shape

(0, 4)

In [10]:
# ********************** Training ******************************* #
import torch
from torch.utils import tensorboard
os.chdir(home)
# feats_dir = os.path.join(home, "features_grid_extract/hm2021/")
feats_dir = "/home/jishengchenmsu/features_grid_extract/hm2021/"
# train_dir = os.path.join(home, "train_500.jsonl") # temporay train 500 images
annotation_dir = "hateful_memes/defaults/annotations/"
dev_unseen_dir = annotation_dir + "dev_unseen.jsonl"
test_unseen_dir = annotation_dir + "test_unseen.jsonl"
train_dir = annotation_dir + "train.jsonl"
ckpt_dir = os.path.join(home, "model6/best.ckpt")

!mmf_run config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
        model="visual_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        checkpoint.resume=True\
        checkpoint.resume_file=$ckpt_dir\
        checkpoint.max_to_keep=1 \
        checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
        training.tensorboard=True \
        training.checkpoint_interval=100 \
        training.evaluation_interval=100 \
        training.max_updates=4000 \
        training.log_interval=500 \
        dataset_config.hateful_memes.max_features=100 \
        dataset_config.hateful_memes.annotations.train[0]=$train_dir \
        dataset_config.hateful_memes.annotations.val[0]=$dev_unseen_dir \
        dataset_config.hateful_memes.annotations.test[0]=$test_unseen_dir \
        dataset_config.hateful_memes.features.train[0]=$feats_dir \
        dataset_config.hateful_memes.features.val[0]=$feats_dir \
        dataset_config.hateful_memes.features.test[0]=$feats_dir \
        training.lr_ratio=0.01 \
        training.use_warmup=True \
        training.batch_size=32 \
        optimizer.params.lr=5.0e-05 \
        env.save_dir=./model6 \
        env.tensorboard_logdir=logs/fit/model6


# ********************** Training ******************************* #
# os.environ['MKL_THREADING_LAYER'] = 'GNU'
# os.chdir(home)
# ckpt_dir = "home/jishengchenmsu/v0_500/best.ckpt"
# feats_dir = os.path.join(home, "features_ocr")
# train_dir = os.path.join(home, "train_500.jsonl")
# annotation_dir = "hateful_memes/defaults/annotations/"
# dev_unseen_dir = annotation_dir + "dev_unseen.jsonl"
# test_unseen_dir = annotation_dir + "test_unseen.jsonl"

# !mmf_run config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
#         model="visual_bert" \
#         dataset=hateful_memes \
#         run_type=train_val \
#         checkpoint.resume_file=$ckpt_dir \
#         checkpoint.reset.optimizer=True \
#         checkpoint.resume=True\
#         checkpoint.max_to_keep=1 \
#         checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
#         training.tensorboard=True \
#         training.checkpoint_interval=50 \
#         training.evaluation_interval=50 \
#         training.max_updates=500 \
#         training.log_interval=100 \
#         dataset_config.hateful_memes.max_features=100 \
#         dataset_config.hateful_memes.annotations.train[0]=$train_dir \
#         dataset_config.hateful_memes.annotations.val[0]=$dev_unseen_dir \
#         dataset_config.hateful_memes.annotations.test[0]=$test_unseen_dir \
#         dataset_config.hateful_memes.features.train[0]=$feats_dir \
#         dataset_config.hateful_memes.features.val[0]=$feats_dir \
#         dataset_config.hateful_memes.features.test[0]=$feats_dir \
#         training.lr_ratio=0.3 \
#         training.use_warmup=True \
#         training.batch_size=32 \
#         optimizer.params.lr=5.0e-05 \
#         env.save_dir=./v0_500 \
#         env.tensorboard_logdir=logs/fit/v0_500\

In [11]:
# import torch
# # PATH = './drive/My Drive/checkpoint.pth'
# PATH = '/content/drive/MyDrive/checkpoint.pth'
# torch.save(model.state_dict(), PATH)
# files.download(PATH)
# state_dict = torch.load(PATH)
# print(state_dict.keys())
# model.load_state_dict(state_dict)

# ********************* baseline predicition **********************
import os
os.chdir(home)
feats_dir = os.path.join(home, "features_ocr")
annotation_dir = "hateful_memes/defaults/annotations/"
dev_unseen_dir = annotation_dir + "dev_unseen.jsonl"
test_unseen_dir = annotation_dir + "test_unseen.jsonl"


!mmf_predict config=projects/hateful_memes/configs/visual_bert/from_coco.yaml \
                model=visual_bert\
                dataset=hateful_memes\
                run_type=test\
                checkpoint.resume_zoo=visual_bert.finetuned.hateful_memes.from_coco\
                dataset_config.hateful_memes.annotations.val[0]=$dev_unseen_dir\
                dataset_config.hateful_memes.annotations.test[0]=$test_unseen_dir\
                dataset_config.hateful_memes.features.train[0]=$feats_dir\
                dataset_config.hateful_memes.features.val[0]=$feats_dir\
                dataset_config.hateful_memes.features.test[0]=$feats_dir

In [ ]:
# # Visualize losses/accuracy via Tensorboard
# # Load the TensorBoard notebook extension
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

In [12]:
# prediction (test_unseen.jsonl)
#************ self-defined model prediction *************************
import os
# # where checkpoint is
os.chdir(home)
ckpt_dir = os.path.join(home, "best.ckpt")
feats_dir = os.path.join(home, "features_ocr")
annotation_dir = "hateful_memes/defaults/annotations/"
dev_unseen_dir = annotation_dir + "dev_unseen.jsonl"
test_unseen_dir = annotation_dir + "test_unseen.jsonl"


!mmf_predict config="projects/visual_bert/configs/hateful_memes/defaults.yaml"\
     model="visual_bert"\
     dataset=hateful_memes\
     run_type=test\
     checkpoint.resume_file=$ckpt_dir\
     checkpoint.reset.optimizer=True\
     dataset_config.hateful_memes.annotations.val[0]=$dev_unseen_dir\
     dataset_config.hateful_memes.annotations.test[0]=$test_unseen_dir\
     dataset_config.hateful_memes.features.train[0]=$feats_dir\
     dataset_config.hateful_memes.features.val[0]=$feats_dir\
     dataset_config.hateful_memes.features.test[0]=$feats_dir

In [ ]:
# # Test pretrained model
# import matplotlib.pyplot as plt
# import requests

# from PIL import Image

# from mmf.models.mmbt import MMBT

# model = MMBT.from_pretrained("mmbt.hateful_memes.images")

In [ ]:
# image_url = "https://i.imgur.com/tEcsk5q.jpg" #@param {type:"string"}
# text = "look how many people love you" #@param {type: "string"}
# output = model.classify(image_url, text)
# plt.imshow(Image.open(requests.get(image_url, stream=True).raw))
# plt.axis("off")
# plt.show()
# hateful = "Yes" if output["label"] == 1 else "No"
# print("Hateful as per the model?", hateful)
# print(f"Model's confidence: {output['confidence'] * 100:.3f}%")